<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/Econophysics/Kinetic_Momentum_Tracker_(KMT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtrader
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backtrader as bt

# Obtenção dos dados
data = yf.download('AAPL', start='2010-01-01', end='2023-01-01')

# Cálculo dos Indicadores
# Posição - Média Móvel Simples
data['SMA'] = data['Close'].rolling(window=20).mean()

# Velocidade - MACD
ema_short = data['Close'].ewm(span=12, adjust=False).mean()
ema_long = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema_short - ema_long

# Aceleração - Variação do MACD
data['MACD_diff'] = data['MACD'].diff()

# Implementação de uma Estratégia de Backtesting com backtrader
class CinematicStrategy(bt.Strategy):
    params = (('sma_period', 20), ('macd_fast', 12), ('macd_slow', 26),)

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_period)
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.macd_fast, period_me2=self.params.macd_slow)
        # Correção: Uso da diferença de valores diretamente
        self.macd_diff = self.macd.macd - self.macd.macd(-1)

    def next(self):
        if not self.position:  # Se não tiver posição
            if self.macd_diff[0] > 0:  # Se a aceleração é positiva
                self.buy(size=100)
        elif self.macd_diff[0] < 0:  # Se a aceleração é negativa
            if self.position:  # Verifica se há uma posição aberta
                self.sell(size=100)

# Configuração do Backtrader e execução (igual ao exemplo anterior)
cerebro = bt.Cerebro()
cerebro.addstrategy(CinematicStrategy)
datafeed = bt.feeds.PandasData(dataname=data)
cerebro.adddata(datafeed)
cerebro.run()
cerebro.plot()
cerebro.plot(style='bar')
plt.savefig('backtest_result.png')  # Salva o gráfico como uma imagem PNG


[*********************100%%**********************]  1 of 1 completed


Este novo indicador resulta da combinação de indicadores técnicos bem conhecidos, como sugerido pela analogia com a cinemática, onde tentamos simular a posição, velocidade e aceleração de um objeto em movimento e aplicar esse conceito ao mercado de ações.

Aqui está a aplicação da analogia:

Posição (x0): Usaremos a Média Móvel Simples (SMA) de um período específico para representar a posição (o preço médio da ação ao longo de um período), sugerindo uma visão geral do comportamento do preço da ação.
Velocidade (v0): Representaremos por meio do MACD, que é uma comparação de duas médias móveis exponenciais (uma rápida e uma lenta), para capturar a direção e a velocidade da mudança de preço, refletindo tendências de curto prazo versus longo prazo.
Aceleração (a_x): Calcularemos a variação do MACD (o segundo diferencial do preço), que poderia representar a aceleração da mudança de preço, indicando como a velocidade da mudança de preço está aumentando ou diminuindo.
Considerações Importantes
Ajuste de Parâmetros: Pode ser necessário ajustar os parâmetros dos indicadores técnicos usados (por exemplo, os períodos das médias móveis do MACD) para alinhar melhor com as características do mercado ou da ação específica.
Análise de Performance: Após o backtesting, é importante analisar a performance da estratégia para entender seu potencial retorno e risco. Métricas como retorno total, drawdown máximo, e o ratio de Sharpe podem ser úteis.